In [1]:
import requests
import pandas as pd
import numpy as np
import xarray as xr
import sqlite3
import os
from datetime import datetime
import cdsapi
import csv

In [2]:
def requete_sql (requete) :
    try:
        connexion = sqlite3.connect('./data/liste_piezos.db')
        curseur = connexion.cursor()
        print("Connexion réussie à SQLite")
        curseur.execute(requete)
        connexion.commit()
        resultat = curseur.fetchall()
        curseur.close()
        connexion.close()
        print("Connexion SQLite est fermée")
        return resultat
    except sqlite3.Error as error:
        print("Erreur lors du mis à jour dans la table : ", error)

# Connexion à la bdd piezo
Construction d'un dataframe 'df_piezo' à partir de la requête sur la bdd

In [9]:
region = "PAC"

In [10]:
connexion = sqlite3.connect('./data/liste_piezos.db')
curseur = connexion.cursor()
requete = f"""
    SELECT code_bss, date_debut, date_fin, nb_mesures_piezo, latitude, longitude 
        FROM data_piezo
    INNER JOIN code_region
    ON code_dpt = departement
    WHERE code_region IS '{region}'
    AND date_debut < '2010-07-01'
    AND date_fin > '2010-07-01'
"""

df_piezo = pd.read_sql_query(requete, connexion, index_col='code_bss')
curseur.close()
connexion.close()

# Conversion des dates
df_piezo["date_debut"] = pd.to_datetime(df_piezo["date_debut"])
df_piezo["date_fin"] = pd.to_datetime(df_piezo["date_fin"])

In [11]:
df_piezo

,date_debut,date_fin,nb_mesures_piezo,latitude,longitude
code_bss,,,,,
09145X0057_N210,2002-01-09,2022-05-22,7248,44.110203,4.751991
09406X0643_P130,2002-09-01,2022-05-23,6643,43.968625,4.830824
09145X0059_N340,2002-09-03,2022-05-19,6792,44.178827,4.735269
09735X0070_P40,2008-06-25,2021-11-30,4397,43.786135,7.217691
08472X0007_F-1,1987-10-01,2022-05-23,7218,44.738691,6.582662
...,...,...,...,...,...
09675X0115_PIEZ,2001-11-20,2022-05-18,6964,43.790683,5.066672
10446X0267_PIEZ,2001-04-10,2022-05-23,6887,43.287758,5.528774
10475X0098_PIEZ,2009-12-08,2022-05-22,3903,43.274616,6.565508


# Fonction pour lancer les requêtes

In [12]:
def get_on_cds_api(dataframe_piezo, piezo, path):
    # Initialisation de la connexion à l'API CDS
    # wait_until_complete = False : le programme Python n'attend pas que l'API réponde (issu de la libraie asyncio)
    # delete = False : le fichier créé sur le serveur source n'est pas supprimé automatiquement 
    
    date_debut = dataframe_piezo.loc[piezo,'date_debut'].year
    date_fin = dataframe_piezo.loc[piezo,'date_fin'].year
    
    # La limite du nombre de données est de 120000. Avec 10 variables, 31 jours, et 12 mois, on n epeut récupérer que 32 années.
    # Si l'interval d'année est supérieur à 32, on réduit la plage de temps à date de fin - 31 ans pour avoir 32 années max.
    if date_fin - date_debut >= 32:
        date_debut = date_fin - 31
    
    c = cdsapi.Client(wait_until_complete=False, delete=False)
    
    # Données à récupérer auprès de l'API
    r = c.retrieve(
            'reanalysis-era5-single-levels',
            {
                'product_type': 'reanalysis',
                'format': 'netcdf',
                'variable': [
                    '2m_temperature', 'convective_precipitation', 'convective_rain_rate',
                    'convective_snowfall', 'large_scale_rain_rate', 'large_scale_snowfall', 'snowfall',
                    'large_scale_snowfall_rate_water_equivalent', 'potential_evaporation',
                    'total_precipitation',
                ],
                'year': [str(date) for date in (np.arange(date_debut, date_fin))],
                'month': [
                    '01', '02', '03',
                    '04', '05', '06',
                    '07', '08', '09',
                    '10', '11', '12',
                ],
                'day': [
                    '01', '02', '03',
                    '04', '05', '06',
                    '07', '08', '09',
                    '10', '11', '12',
                    '13', '14', '15',
                    '16', '17', '18',
                    '19', '20', '21',
                    '22', '23', '24',
                    '25', '26', '27',
                    '28', '29', '30',
                    '31',
                ],
                'time': '00:00',
                'area': [
                    df_piezo.loc[piezo,"latitude"], df_piezo.loc[piezo,"longitude"], df_piezo.loc[piezo,"latitude"], df_piezo.loc[piezo,"longitude"],
                ],
            })    
    return r

## Lancer les requêtes auprès de l'API
!!! NE LANCER QU'UNE SEULE FOIS POUR NE PAS CREER DES DOUBLONS DE REQUETES !!!

Un dictionnaire est créé avec en clé le code_bss du piezo et en valeur l'id de la requête

Ne pas lancer la cellule ci-dessous si des requetes sont en cours. 

In [13]:
# Parcourir les codes_bss des piezos dans la liste précédemment créé :
dict_bss = {}

In [14]:
go_requete = input("""
            Voulez-vous lancer une série de nouvelles requêtes ? /n 
            Des requêtes pourraient se retrouver en double. /n 
            Pour lancer les requêtes entrez 'req' et entrée. /n
            Sinon, appuyez simplement sur entrée.
""")
                   
if go_requete == 'req' :
                      
    for piezo in df_piezo.index:
        path_piezo = 'data/era5/' + piezo
        path_nc = f"./{path_piezo}.nc"
        path_pkl = f"{path_piezo}.pkl"

        if os.path.isfile(path_nc) : #si le fichier nc existe on le converti en pkl
            print(f"Le fichier {path_nc} existe.")
            df = xr.open_dataset(path_nc).to_dataframe().reset_index().to_pickle(path=path_pkl)
            print(f"Fichier {piezo}.nc converti en fichier pickle")
            os.remove(path_nc)
            print(f"Fichier {path_nc} supprimé")

        elif os.path.isfile(path_pkl) : #si le fichier pkl existe, on affiche qu'il existe
            print(f"Le fichier {piezo}.pkl existe déjà.")
            # puis on le supprime de la liste
            df_piezo.drop(index=piezo, axis=1)

        else : # Sinon on lance une requête auprès de l'API
            result = get_on_cds_api(df_piezo, piezo, path_nc)
            # récupération de l'id de la requête
            dict_bss[piezo] = result.reply["request_id"]
            print(f"{datetime.now()} : chargement du fichier {piezo}.nc en cours ({dict_bss[piezo]}).")
            
            # Enregistrement de l'id de requete dans un fichier csv
            if os.path.isfile('./data/hist_requetes.csv'): # Si le fichier existe                
                with open('./data/hist_requetes.csv', 'a', encoding='utf-8', newline='') as f:  
                    writer = csv.writer(f, delimiter = ";")
                    for k, v in dict_bss.items():
                        writer.writerow((k, v))
            else : # Si le fichier n'existe pas
                with open('./data/hist_requetes.csv', 'w', encoding='utf-8', newline='') as f:  
                    writer = csv.writer(f, delimiter = ";")
                    writer.writerow(["code_bss", "id_requete"])
                    for k, v in dict_bss.items():
                        writer.writerow((k, v))
else : 
    print("Aucune requête n'a été lancée.")


            Voulez-vous lancer une série de nouvelles requêtes ? /n 
            Des requêtes pourraient se retrouver en double. /n 
            Pour lancer les requêtes entrez 'req' et entrée. /n
            Sinon, appuyez simplement sur entrée.
req


2022-07-25 13:17:50,970 INFO Welcome to the CDS
2022-07-25 13:17:50,970 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2022-07-25 13:17:51,115 INFO Welcome to the CDS
2022-07-25 13:17:51,116 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2022-07-25 13:17:51,240 INFO Welcome to the CDS


2022-07-25 13:17:51.060285 : chargement du fichier 09145X0057_N210.nc en cours (8920aaf5-1c5a-4add-82ed-67c578423c7a).
2022-07-25 13:17:51.178780 : chargement du fichier 09406X0643_P130.nc en cours (37b83f68-a036-48f6-bea6-33ef2cab27e4).


2022-07-25 13:17:51,240 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2022-07-25 13:17:51,356 INFO Welcome to the CDS
2022-07-25 13:17:51,356 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2022-07-25 13:17:51,491 INFO Welcome to the CDS


2022-07-25 13:17:51.309218 : chargement du fichier 09145X0059_N340.nc en cours (f82677bd-9e72-4719-a6b6-34952bcfb414).
2022-07-25 13:17:51.425170 : chargement du fichier 09735X0070_P40.nc en cours (76c7cec4-9a15-46a5-8e1d-71f7b3f44100).
Le fichier 08472X0007_F-1.pkl existe déjà.


...

2022-07-25 13:18:03.626495 : chargement du fichier 09695X0205_PIEZ.nc en cours (90d2b2ff-cec2-4167-b39e-e0ebf8ba85d7).
2022-07-25 13:18:03.740933 : chargement du fichier 09675X0115_PIEZ.nc en cours (bed2e3ec-58bc-4af1-9a5b-643e1e94bfcb).
2022-07-25 13:18:03.869309 : chargement du fichier 10446X0267_PIEZ.nc en cours (8ff6c84e-fd79-4421-9e3b-a8b6071f00c1).


2022-07-25 13:18:04,125 INFO Welcome to the CDS
2022-07-25 13:18:04,126 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2022-07-25 13:18:04,259 INFO Welcome to the CDS
2022-07-25 13:18:04,260 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2022-07-25 13:18:04,385 INFO Welcome to the CDS


2022-07-25 13:18:04.186949 : chargement du fichier 10475X0098_PIEZ.nc en cours (e957759d-003c-48bf-a53c-c2951616c9ca).
2022-07-25 13:18:04.312253 : chargement du fichier 10195X0041_PZ.nc en cours (162f9b71-2c70-479e-8a71-be3b47b0a2fe).


2022-07-25 13:18:04,386 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels


2022-07-25 13:18:04.455332 : chargement du fichier 09924X0114_PZ.nc en cours (aa35d41d-1943-4dea-bbcf-9825b8cd6f71).


## Récupérer automatiquement les requêtes abouties 
Peut être lancé à tout moment pour récupérer des fichiers téléchargés 

In [7]:
# Chargement du dictionnaire à partir du csv d'historiques de requetes
with open('./data/hist_requetes.csv', mode='r') as f:
    reader = csv.reader(f, delimiter=";")
    # Permet de sortir la ligne d'en-tête du dictionnaire
    headings = next(reader)
    dict_bss_from_csv = {rows[0]:rows[1] for rows in reader}

nbre_fichier_OK = 0

    
for piezo in dict_bss_from_csv.copy().keys():
    path_piezo = 'data/era5/' + piezo
    path_nc = f"./{path_piezo}.nc"
    path_pkl = f"{path_piezo}.pkl"

    if not os.path.isfile(path_nc) :
        try:
            reply = dict(request_id=dict_bss_from_csv[piezo]) # request_id from above
            new_client = cdsapi.Client(wait_until_complete=False, delete=False)
            result = cdsapi.api.Result(new_client, reply)
            result.update()
            reply = result.reply
            if reply['state'] == 'completed':
                nbre_fichier_OK += 1
                result.download(path_nc)
                print(f"{datetime.now()} : Fichier {piezo}.nc téléchargé.")
                df = xr.open_dataset(path_nc).to_dataframe().reset_index().to_pickle(path=path_pkl)
                print(f"Fichier {piezo}.nc converti en fichier pickle")
                os.remove(path_nc)
                print(f"Fichier {path_nc}.nc supprimé")
                # Si le fichier a été téléchargé on le supprime de la liste du dictionnaire
                del dict_bss[piezo]
        except:
            #print(f"Récupération de l'ID {dict_bss_from_csv[piezo]} impossible.")
            pass
if nbre_fichier_OK == 0:
    print("Aucun fichier de récupéré.")
else :
    print(f"{nbre_fichier_OK} fichiers ont été téléchargés.")

# Mise à jour de la base de donnéees (champ fichier_ERA5)
bss_era5 = pd.Series([_.replace(".pkl","") for _ in os.listdir(r"./data/era5") if _.endswith(r".pkl")], name="era5")
requete_fichier_era5 = f"""
        UPDATE data_piezo
        SET fichier_era5 =
        (CASE WHEN code_bss IN {tuple(bss_era5)}
        THEN 1 ELSE 0 END)
        """
requete_sql(requete_fichier_era5)
print("Base de données à jour")

2022-07-25 13:04:16,167 INFO Downloading https://download-0016.copernicus-climate.eu/cache-compute-0016/cache/data4/adaptor.mars.internal-1658483723.6151054-32144-5-a338c757-0538-421c-90f3-af058578b4e1.nc to ./data/era5/09306X0286_F.nc (200.3K)
2022-07-25 13:04:16,598 INFO Downloading https://download-0002-clone.copernicus-climate.eu/cache-compute-0002/cache/data2/adaptor.mars.internal-1658483766.8941596-31819-16-1b6e349f-a2b5-41d9-b58e-4a9ad57ad256.nc to ./data/era5/09838D0727_F.nc (217.4K)
2022-07-25 13:04:16,869 INFO Downloading https://download-0007-clone.copernicus-climate.eu/cache-compute-0007/cache/data2/adaptor.mars.internal-1658483723.540129-25749-4-c569fdaa-c4d1-4b45-b20b-e2fd7695da51.nc to ./data/era5/09577X0047_F.nc (226K)
2022-07-25 13:04:17,535 INFO Downloading https://download-0007-clone.copernicus-climate.eu/cache-compute-0007/cache/data2/adaptor.mars.internal-1658483723.6680422-22280-17-b9c5706a-4636-4821-b3a2-635aec36c922.nc to ./data/era5/09302X0139_F.nc (200.3K)
202

2022-07-25 13:04:21.375412 : Fichier 09394X0003_FONTAI.nc téléchargé.
Fichier 09394X0003_FONTAI.nc converti en fichier pickle
Fichier ./data/era5/09394X0003_FONTAI.nc.nc supprimé


2022-07-25 13:04:21,819 INFO Downloading https://download-0021.copernicus-climate.eu/cache-compute-0021/cache/data6/adaptor.mars.internal-1658493038.9171891-25959-8-2feaf093-ea62-4c34-bab0-c886baa44e6d.nc to ./data/era5/09128X0053_CAL.nc (149K)
2022-07-25 13:04:22,326 INFO Downloading https://download-0013-clone.copernicus-climate.eu/cache-compute-0013/cache/data0/adaptor.mars.internal-1658493034.177106-20713-8-55fdb396-6240-402c-84be-4b155b6527c9.nc to ./data/era5/08622X0027_GAL.nc (123.3K)
2022-07-25 13:04:22,638 INFO Downloading https://download-0013-clone.copernicus-climate.eu/cache-compute-0013/cache/data9/adaptor.mars.internal-1658494640.7272131-2629-12-464f5783-7d40-4081-8d59-6cd6eb31c1d6.nc to ./data/era5/09905X0064_CMBSAL.nc (131.9K)
2022-07-25 13:04:22,948 INFO Downloading https://download-0013-clone.copernicus-climate.eu/cache-compute-0013/cache/data1/adaptor.mars.internal-1658495913.0423882-2516-15-ba9987b8-11ba-4cb6-96ad-b0731b111dfe.nc to ./data/era5/10764X0043_OUSTAL.nc 

2022-07-25 13:04:24.010509 : Fichier 09656X0175_RODIL.nc téléchargé.
Fichier 09656X0175_RODIL.nc converti en fichier pickle
Fichier ./data/era5/09656X0175_RODIL.nc.nc supprimé


2022-07-25 13:04:24,346 INFO Downloading https://download-0000-clone.copernicus-climate.eu/cache-compute-0000/cache/data9/adaptor.mars.internal-1658495906.3505707-29805-13-c3531de1-e67f-479a-85c8-07c419ec3740.nc to ./data/era5/09398X0028_VALL.nc (157.5K)
2022-07-25 13:04:24,464 INFO Downloading https://download-0000-clone.copernicus-climate.eu/cache-compute-0000/cache/data8/adaptor.mars.internal-1658494407.0805523-32634-8-0d582f01-ae36-47bd-bfe0-45686b22897d.nc to ./data/era5/10406X0077_ASL.nc (131.9K)
2022-07-25 13:04:24,743 INFO Downloading https://download-0011-clone.copernicus-climate.eu/cache-compute-0011/cache/data5/adaptor.mars.internal-1658493028.9011672-7270-5-5b6cd4db-9477-408e-b3a2-ed653b3fe0b6.nc to ./data/era5/10904X0105_ROBOUL.nc (174.6K)
2022-07-25 13:04:25,176 INFO Download rate 403.9K/s
2022-07-25 13:04:25,257 INFO Downloading https://download-0014-clone.copernicus-climate.eu/cache-compute-0014/cache/data5/adaptor.mars.internal-1658498894.0992043-25722-9-0a88ef78-e103-

...

2022-07-25 13:04:42.941881 : Fichier 09388X0052_VIGNOT.nc téléchargé.
Fichier 09388X0052_VIGNOT.nc converti en fichier pickle
Fichier ./data/era5/09388X0052_VIGNOT.nc.nc supprimé


2022-07-25 13:04:43,275 INFO Downloading https://download-0020.copernicus-climate.eu/cache-compute-0020/cache/data3/adaptor.mars.internal-1658526739.7026966-11882-5-145f0222-9d0e-42ee-9382-14d02bb23215.nc to ./data/era5/09876X0212_PORT.nc (149K)
2022-07-25 13:04:43,560 INFO Downloading https://download-0015-clone.copernicus-climate.eu/cache-compute-0015/cache/data6/adaptor.mars.internal-1658526739.9242733-5826-17-83be31c3-1393-40cb-a07d-fb99def1ed8b.nc to ./data/era5/10976X0058_RIMBAU.nc (157.5K)
2022-07-25 13:04:43,882 INFO Downloading https://download-0015-clone.copernicus-climate.eu/cache-compute-0015/cache/data6/adaptor.mars.internal-1658526341.13916-17470-11-348000ea-9bf6-4c27-ab1c-bb227aaa0aba.nc to ./data/era5/09882X0208_FONCAU.nc (149K)
2022-07-25 13:04:44,219 INFO Downloading https://download-0001-clone.copernicus-climate.eu/cache-compute-0001/cache/data0/adaptor.mars.internal-1658527615.0461516-7109-4-8c140dd6-f2a8-4e50-9a8c-3c3be8481160.nc to ./data/era5/09908X0174_MASJAU.nc

2022-07-25 13:04:46.291886 : Fichier BSS003PAFM_X.nc téléchargé.
Fichier BSS003PAFM_X.nc converti en fichier pickle
Fichier ./data/era5/BSS003PAFM_X.nc.nc supprimé


2022-07-25 13:04:46,698 INFO Downloading https://download-0007-clone.copernicus-climate.eu/cache-compute-0007/cache/data7/adaptor.mars.internal-1658521939.2463582-14574-15-0fbe163b-302a-4522-970b-ba4bdf9f839f.nc to ./data/era5/10882X0215_ADOUX.nc (106.2K)
2022-07-25 13:04:47,028 INFO Downloading https://download-0003-clone.copernicus-climate.eu/cache-compute-0003/cache/data0/adaptor.mars.internal-1658521213.204858-31409-7-24fe87a3-85d3-49f4-837e-8ce4fbd41b88.nc to ./data/era5/09625X0231_LADOUX.nc (149K)
2022-07-25 13:04:47,357 INFO Downloading https://download-0017.copernicus-climate.eu/cache-compute-0017/cache/data4/adaptor.mars.internal-1658528314.053892-18054-10-4a2ab5bd-d692-4ed0-b773-9d7f34422098.nc to ./data/era5/09126X0275_RABY.nc (106.2K)
2022-07-25 13:04:47,674 INFO Downloading https://download-0020.copernicus-climate.eu/cache-compute-0020/cache/data2/adaptor.mars.internal-1658528077.291807-18398-11-f8d9c845-f14f-4581-b019-8317ef8efcc4.nc to ./data/era5/08576X0022_HY.nc (89K)


95 fichiers ont été téléchargés.
Connexion réussie à SQLite
Connexion SQLite est fermée
Base de données à jour


## Jointure era5 et piezo

In [8]:
bss_era5 = pd.Series([_.replace(".pkl","") for _ in os.listdir(r"./data/era5") if _.endswith(r".pkl")], name="era5")
bss_piezo = pd.Series([_.replace(".csv","") for _ in os.listdir(r"./data/piezo") if _.endswith(r".csv")], name="piezo")
piezos_communs = np.intersect1d(bss_era5, bss_piezo)

path = "./data/combined_data/"
# Le fichier doit avoir un minimum de 100 mesures
nb_min_val_com = 100

for piezo in piezos_communs :
    if not os.path.isfile(f'{path}/comb_{piezo}.csv') : #si le fichier n'existe pas 
        df_piezo = pd.read_csv(f"./data/piezo/{piezo}.csv", sep = ";", index_col=0, parse_dates=True)
        # Group_by date_mesure dans le cas de plusieurs mesures le même jour ce qui provoquerait une erreur
        df_piezo = pd.DataFrame(df_piezo.groupby("date_mesure").mean())
        df_piezo = df_piezo.resample('1D').asfreq()
        df_era5 = pd.read_pickle(f"./data/era5/{piezo}.pkl").set_index('time').drop(["latitude", "longitude"], axis=1)
        if "expver" in df_era5.columns :
            df_era5 = df_era5.loc[df_era5["expver"]==1,:].drop("expver", axis=1)
        df = pd.merge(df_piezo, df_era5, how='inner', left_index=True, right_index=True)
        nb_com_obs = df.shape[0]
        if nb_com_obs >= nb_min_val_com :
            df.to_csv(f"data/combined_data/comb_{piezo}.csv" ,sep=";", index=True)
            print(f"Le fichier comb_{piezo}.csv a été créé et contient {nb_com_obs} obseravtions.")
        else :
            print(f"Le piezo {piezo} ne contient pas assez de données (seulement {nb_com_obs}).")
    else : 
        #print(f"Le fichier comb_{piezo}.csv existe déjà.")
        pass
        
# Mise à jour bdd
requete_fichier_combined = f"""
        UPDATE data_piezo
        SET fichier_combined =
        (CASE WHEN code_bss IN {tuple(piezos_communs)}
        THEN 1 ELSE 0 END)
        """
requete_sql(requete_fichier_combined)
print("Base de données à jour")

Connexion réussie à SQLite
Connexion SQLite est fermée
Base de données à jour


# Récupération manuelle

In [ ]:
reply = dict(request_id="54c52ee3-eee4-481d-b72e-fecd315085cb")
new_client = cdsapi.Client(wait_until_complete=False, delete=False)
result = cdsapi.api.Result(new_client, reply)
result.update()
reply = result.reply
if reply['state'] == 'completed':
    result.download('file.nc')

In [ ]:
requete_fichier_era5 = f"""
        UPDATE data_piezo
        SET fichier_era5 =
        (CASE WHEN code_bss IN {tuple(bss_era5)}
        THEN 1 ELSE 0 END)
        """
requete_sql(requete_fichier_era5)

In [ ]:
update table set
    columnx = (case when condition then 25 else columnx end)